In [2]:
### 1. Data loading

In [3]:
import os
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import  pickle
import gc
import time
import re
from tqdm import tqdm

import optuna
from optuna import distributions
import logging
import warnings

import torch
from torch.utils.data import WeightedRandomSampler
from torch.utils.data import Dataset, DataLoader
from torch import nn
import torch.nn.functional as F
import torch.optim as optim

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import f1_score
from sklearn.preprocessing import( StandardScaler,
                                   MinMaxScaler,
                                   OneHotEncoder,
                                   OrdinalEncoder              
)
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer 

from collections import Counter
from category_encoders import HashingEncoder, TargetEncoder

RANDOM_STATE = 51

### 2. Data preprocessing

In [4]:

def missing(data):
    missing = pd.DataFrame(data.isna().sum(),columns=["missing_values"]).reset_index().query('missing_values != 0 ')
    missing.rename(columns={'index': 'column_name'}, inplace=True)
    return missing.reset_index(drop=True)
    

def score( y_true,y_pred):
    print('accuracy: ',accuracy_score( y_true, y_pred))
    print('precision: ', precision_score( y_true, y_pred))
    print( 'recall: ', recall_score( y_true, y_pred))
    print('f1: ',f1_score(y_true, y_pred))          


In [5]:
try:
    path_data_train= Path(Path.cwd()/'adengi-internship_1/train.csv')
    path_data_test= Path(Path.cwd()/'adengi-internship_1/test.csv')
   
    df_train = pd.read_csv(path_data_train)
    df_test = pd.read_csv(path_data_test)
    
    
except:

    path_data_train = Path('/kaggle/input/a-money/train.csv')
    path_data_test = Path('/kaggle/input/a-money/test.csv')

    df_train = pd.read_csv(path_data_train)
    df_test = pd.read_csv(path_data_test)
    
print(path_data_train, '\n',path_data_test)


/kaggle/input/a-money/train.csv 
 /kaggle/input/a-money/test.csv


In [6]:
replace_dict = {
    'чуваш': 'Чувашская',
    'мари': 'Марий Эл',
     'луган': 'ЛНР',
    'донец': 'ДНР',
    'якут': 'Якутия',
    'осет':'Северная Осетия',
    'саха' : 'Якутия'
    
}


replace_dict_2  = {
    'ямал':  'ао Ямало-Ненецкий',
    'ненец': 'ао Ненецкий',
    'ханты' : 'ао Ханты-Мансийский',
     
    'чукотск':'ао Чукотский', 
    'еврейск': 'аобл Еврейская'
    
}

replace_dict_3 = {
    'москв':'обл Московская',
    'севаст':'респ Крым',
    ' санкт':'обл Ленинградская'
}
def change(text):
    text =' '.join(text)
    string= False
    for i in replace_dict:
        if  re.search( rf'(?i)\w*{i}\w*', text):
            string = replace_dict.get(i)
    if  string:
        return 'респ '+string
    return 'респ ' + text   

def  change_2(text): 
    for i in replace_dict_2:
        if re.search( rf'(?i)\w*{i}\w*', text):
            return replace_dict_2.get(i) 

def  change_3(text): 
    for i in replace_dict_3:
        if re.search( rf'(?i)\w*{i}\w*', text):
            return replace_dict_3.get(i) 
      

def region_1(string):
    # string = string.astype('str')
    if re.search(r'(?i)\bобл\b|\bобласть\b', string):
        # print('обл')
        return 'обл '+set(re.findall(r'\b\w*кая\b', string)).pop().capitalize()
    #elif re.search(r'\bг\b|\bрайон\b', string):
        # print('г')
        #return 'г '+ set(re.sub(r'(?i)\bг\b|\bрайон\b' ,'', string).split()).pop()
    elif re.search(r'\bг\b|\bрайон\b', string):
        #print('г')
        return change_3(string)
    
    elif  re.search(r'(?i)респ', string):
        # print('респ')
        resp = list(set(re.sub(r'(?i)\b\w*респ\w*\b', '', string).split()))
        return change(resp) 
    elif re.search(r'(?i)край',string) and not re.search(r'(?i)область', string):
        return 'край '+set(re.findall(r'\b\w*ский\b', string)).pop().capitalize()

    elif re.search(r'(?i)\bао\b|\bавтономный\b|\bеврейская\b', string):
        # print('AO',string, 'sss')
        return change_2(string)
    return 'unknown ' + string

In [7]:
%%time
df_train['region_new']= df_train['region'].map(region_1)
df_test['region_new'] = df_test['region'].map(region_1)

CPU times: user 33.7 s, sys: 311 ms, total: 34 s
Wall time: 34 s


In [8]:
region_train= df_train['region_new'].unique()
region_test = df_test['region_new'].unique()
print(f'Число регионов в test: {len(region_test)},', f'Число регионов в train: {len(region_train)}')

print('дельта test^train: ',set(df_test['region_new'])^set( df_train['region_new']))

Число регионов в test: 86, Число регионов в train: 86
дельта test^train:  set()


In [9]:
print('settlement unique(): ', len(df_train['settlement'].unique()))

N=7000 #  частота <10000 закодируем other
settlement= Counter(df_train['settlement'])
vocab_settelement = [i[0] for i in settlement.most_common(N)] 

def settlement_columns(data):
    #data['settlement_new'] =data['settlement'].apply(lambda x: x.lower() if x in vocab_settelement else 'others')   
    data['settlement_new'] =data['settlement'].where(data['settlement'].isin(vocab_settelement),'others')

settlement unique():  47357


In [10]:
%%time
settlement_columns(df_train)
settlement_columns(df_test)

CPU times: user 532 ms, sys: 2.01 ms, total: 534 ms
Wall time: 533 ms


In [11]:
def time_columns (data):
    data['created_at'] = pd.to_datetime(data['created_at'], errors='coerce')
    data['start_dt'] = pd.to_datetime(data['start_dt'], errors='coerce')
    
    data['start_dt_new'] = data['start_dt'].copy()  # для  реализации п. 2.1.3.1 в baseline не используется
     
    data['start_dt'] = data.groupby('client_id')['start_dt'].transform(
            lambda x: x.fillna(x.min()))
    
    data['start_dt'] = data['start_dt'].fillna(data['created_at'])

    data['month'] = data['created_at'].dt.month
    data['day'] = data['created_at'].dt.day
    data['week'] = data['created_at'].dt.dayofweek

    reference_date = pd.to_datetime('2022-01-01')
    
    data['created_at'] = (data['created_at'] - reference_date).dt.days
    data['start_dt'] = (data['start_dt'] - reference_date).dt.days
    
    data['days_diff'] = data['start_dt']-data['created_at']

    data['start_dt_xg'] =  pd.to_datetime(data['start_dt_new'])
    data['start_dt_xg'] =(data['start_dt_xg']-reference_date).dt.days

In [12]:
%%time
time_columns(df_train)
time_columns(df_test)

CPU times: user 5min 30s, sys: 4.83 s, total: 5min 34s
Wall time: 5min 35s


_____________________

In [13]:
df_train.select_dtypes(include='float').columns

Index(['monthly_income', 'work_experience', 'requested_sum',
       'main_agreement_amount', 'main_agreement_term', 'requested_period_days',
       'requested_amount', 'req_app_amount', 'approved_amount', 'period_days',
       'days_finish_loan', 'ag', 'cnt_ext', 'term', 'price', 'elecs_sum',
       'recurents_sum', 'tamount', 'issues', 'principal', 'interest',
       'overdue_interest', 'overdue_fee', 'nbki_score', 'contact_cases',
       'start_dt_xg'],
      dtype='object')

- monthly_income - среднемесячный заработок клиента (зарплата)
- work_experience - кол-во лет стажа клиента
- requested_sum - запрашиваемая сумма клиента для займа, если interface - alfa
- requested_period_days - запрашиваемый срок по займу
- requested_amount - запрашиваемая сумма клиента по займу
- req_app_amount - разница между запрашиваемой суммой займа и одобренной
- cnt_ext - кол-во пролонгаций по займу
- term - срок пролонгации (список)
- price - цена пролонгации (список)
- elecs_sum - штрафы, пени
- recurents_sum - штрафы, пени (там вроде как различия в этапах их начисления)
- tamount - общий кэшфлоу клиента (общая сумма которая была на аккаунте клиента)
- nbki_score - скор клиента от рисков
- contact_cases - кол-во обращений клиента с коллекшн

In [14]:
df_test.loc[df_test['monthly_income']>10,'monthly_income'] =\
    df_test.loc[df_test['monthly_income']>10,'monthly_income']/1000

df_train.loc[df_train['monthly_income']>10,'monthly_income'] =\
    df_train.loc[df_train['monthly_income']>10,'monthly_income']/1000

In [15]:
def simpl_median(name):
    imp_median = SimpleImputer(strategy='median')
    df_train[name] = imp_median.fit_transform(df_train[[name]]) 
    df_test[name] = imp_median.transform(df_test[[name]])

In [16]:
simpl_median('monthly_income')

In [17]:
def min_simple(name):
    df_train[name]=df_train[name].fillna(df_train[name].min())
    df_test[name]=df_test[name].fillna(df_test[name].min())

In [18]:
df_train['cnt_ext'].min(), df_test['cnt_ext'].min()

(-0.7206081182984045, -0.7206081182984045)

In [19]:
min_simple('cnt_ext')

In [20]:
def zerro_simple(name):
    df_train[name]=df_train[name].fillna(0)
    df_test[name]=df_test[name].fillna(0)

In [21]:
zerro_simple('contact_cases')

In [22]:
min_simple('elecs_sum')
min_simple('recurents_sum')

In [23]:
simpl_median('nbki_score')

In [24]:
min_simple('price')

In [25]:
df_train['requested_sum'] = df_train['requested_sum'].fillna(
            df_train['approved_amount'] * df_train['requested_sum'].mean() / df_train['approved_amount'].mean())

df_test['requested_sum'] = df_test['requested_sum'].fillna(
            df_test['approved_amount'] * df_test['requested_sum'].mean() / df_test['approved_amount'].mean())

In [26]:
df_train['requested_amount'] = df_train['requested_amount'].fillna(df_train['requested_sum'])
df_test['requested_amount'] =  df_test['requested_amount'].fillna(df_test['requested_sum'])

In [27]:
df_train['req_app_amount'] = df_train['req_app_amount'].fillna(df_train['requested_amount'] - df_train['approved_amount'])
df_test['req_app_amount'] =df_test['req_app_amount'].fillna( df_test['requested_amount'] - df_test['approved_amount'])

In [28]:
simpl_median('requested_period_days')

In [29]:
def simple_moda(name):
    imp_mode = SimpleImputer(strategy="most_frequent")
    df_train[name]=imp_mode.fit_transform(df_train[[name]])
    df_test[name]=imp_mode.transform(df_test[[name]])

In [30]:
simple_moda('tamount')

In [31]:
zerro_simple('work_experience')

In [32]:
min_simple('term')

In [33]:
columns_flat_no_na=['ag',
'approved_amount',
'days_finish_loan',
'interest',
'issues',
'main_agreement_amount',
'main_agreement_term',
'overdue_fee',
'overdue_interest',
'period_days',
'principal']

In [34]:
columns_int = df_train.select_dtypes(include=['int64', 'int32']).columns
columns_int

Index(['payment_frequency', 'status', 'loan_id', 'client_id', 'source',
       'first_source', 'interface', 'type', 'repayment_type', 'client_type',
       'loan_order', 'have_extension', 'created_at', 'start_dt', 'churn',
       'month', 'day', 'week'],
      dtype='object')

- payment_frequency - частота получения зарплаты (month - 1 раз в месяц, 2 weeks - раз в две недели, и тд)
- status - статус клиента (самозанятый, рабочий, и тд)
- source - канал привлечения клиента
- first_source - первый канал привлечения клиента
- interface - интерфейс, откуда пришла заявка - (site, mobile)
- type - тип займа (тип продукта)
- repayment_type - Тип комиссии по займу (с 2.5% - with_comission, 5% - with_big_comission, 0% - no_comission)
- client_type - тип клиента (новый, повторный)
- have_extension - имеется ли пролонгация по данному займу

In [35]:
list_int= ['payment_frequency', 'status','source', 'first_source',
         'interface', 'type', 'repayment_type', 'client_type','have_extension']

In [36]:
client_count=dict(Counter(pd.concat([df_train['client_id'],df_test['client_id']])))

In [37]:
df_train['client_id_new'] = df_train['client_id'].transform(lambda x: client_count.get(x)) 
df_test['client_id_new'] = df_test['client_id'].transform(lambda x: client_count.get(x)) 

In [38]:
client_count=dict(Counter(pd.concat([df_train['loan_order'],df_test['loan_order']])))

In [39]:
df_train['loan_order_new'] = df_train['loan_order'].transform(lambda x: client_count.get(x)) 
df_test['loan_order_new'] = df_test['loan_order'].transform(lambda x: client_count.get(x)) 

### 3. New features

In [40]:
df_train['early_repayment'] = df_train['period_days'] - df_train['days_finish_loan']
df_test['early_repayment'] = df_test['period_days'] - df_test['days_finish_loan']

In [41]:
 df_train['approval_ratio'] = df_train['approved_amount'] / df_train['requested_amount']
 df_test['approval_ratio'] = df_test['approved_amount'] / df_test['requested_amount']

In [42]:
df_train['requsted_income_ratio'] = df_train['requested_amount']/df_train['monthly_income']
df_test['requsted_income_ratio'] = df_test['requested_amount']/df_test['monthly_income']

In [43]:
df_train['total_debt_burden'] = (df_train['elecs_sum'] + df_train['recurents_sum']) / df_train['monthly_income']
df_test['total_debt_burden'] = (df_test['elecs_sum'] + df_test['recurents_sum']) / df_test['monthly_income']

In [44]:
df_train['dti'] = df_train['principal'] / df_train['monthly_income']
df_test['dti'] = df_test['principal']/ df_test['monthly_income']

In [45]:
df_train['requested_approved_ratio'] = df_train['requested_period_days'] / df_train['main_agreement_term']
df_test['requested_approved_ratio'] =  df_test['requested_period_days'] / df_test['main_agreement_term']

In [46]:
df_train[df_train.select_dtypes(include='int64').columns]=df_train.select_dtypes(include='int64').astype('int32')
df_test[df_test.select_dtypes(include='int64').columns]=df_test.select_dtypes(include='int64').astype('int32')

### 4. Data preparation

In [47]:
list_id_total = ['client_id_new', 'loan_order_new']
list_id = ['loan_id']
list_client= ['monthly_income','nbki_score','tamount','work_experience','ag','payment_frequency',
              'status','client_type','settlement_new','region_new','gender','client_type.1' ]

list_loan= ['cnt_ext','elecs_sum','recurents_sum','price','requested_amount',
            'req_app_amount','term','approved_amount','interest',
            'issues','main_agreement_amount','overdue_fee','overdue_interest','principal','type', 
             'repayment_type','have_extension','requested_sum']	
list_new = ['approval_ratio','early_repayment','requested_approved_ratio','requsted_income_ratio','total_debt_burden','dti']

list_support =[ 'contact_cases','source','first_source','interface'] 
list_time = [ 'month','day','week','days_finish_loan', 'period_days', 
             'requested_period_days','main_agreement_term','created_at','days_diff','start_dt']    
# 'closed_at','prolong','period', 'start_dt'->del

# проверка на дубли
total_list =list_id_total +list_loan+list_client+list_support+list_time + list_new
print('Дубли True-нет: ',len(total_list) == len (set(total_list)))
      
# проверка на идентичность структуры train test
print('Идентичность train, test: ',set(df_train.columns) ^set(df_test.columns))
# проверка на полноту  включения столбцов  в списки
print('Полнота столбцов в списке(:',set(df_train.columns) ^set(total_list))     

Дубли True-нет:  True
Идентичность train, test:  {'churn'}
Полнота столбцов в списке(: {'region', 'settlement', 'loan_order', 'client_id', 'start_dt_xg', 'loan_id', 'start_dt_new', 'churn', 'closed_at'}


In [48]:
display(missing (df_train[total_list]),missing (df_test[total_list]))

,column_name,missing_values


,column_name,missing_values


Проведем поиск  столбцов, для которых множества значений в test и train  равны.
Такие  признаки  можно рассматривать как категориальные

In [49]:
prepocessing = {'region_new': 'one_hot',
 'settlement_new': 'one_hot',
 'created_at': 'scale',
 ' start_dt': 'scale',
 'days_diff': 'scale',
 'start_dt_xg': 'scale',
 'client_type.1': 'one_hot',
 'gender': 'one_hot',
 'contact_cases': 'scale',
 'payment_frequency': 'one_hot',
 'status': 'one_hot',
 'source': 'one_hot',
 'first_source': 'one_hot',
 'interface': 'one_hot',
 'type': 'one_hot',
 'repayment_type': 'one_hot',
 'client_type': 'one_hot',
 'have_extension': 'one_hot',
 'client_id_new': 'scale',
 'loan_order_new': 'scale',
 'early_repayment': 'scale',
 'approval_ratio': 'scale',
 'requsted_income_ratio': 'scale',
 'total_debt_burden': 'scale',
 'requested_approved_ratio': 'scale',
 'dti': 'scale'}

### 4.  Build the Neural Network and train

In [50]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [51]:
print(df_train[total_list].select_dtypes(include='object').columns)

svc_scale_list = list(set(df_train[total_list].select_dtypes(include='float64').columns))+['client_id_new', 'loan_order_new']+\
                ['month', 'day', 'week', 'created_at', 'days_diff',  'start_dt']

svc_oneh_list = list_int + ['client_type.1', 'gender'] 
svc_has_list = ['region_new','settlement_new']

print('Уникальность элементов: ',len(set(svc_scale_list+svc_oneh_list+svc_has_list ))== len(svc_scale_list+svc_oneh_list+svc_has_list))
print('эквивалентность списков:' ,set(svc_scale_list+svc_oneh_list+svc_has_list )^set(total_list))

Index(['settlement_new', 'region_new', 'gender', 'client_type.1'], dtype='object')
Уникальность элементов:  True
эквивалентность списков: set()


In [52]:
X_train, X_test, y_train, y_test =train_test_split(df_train[total_list],df_train['churn'], test_size=0.20, random_state=RANDOM_STATE, stratify=df_train['churn'])

In [53]:
preprocessing =ColumnTransformer([
    ('scaler', StandardScaler(),svc_scale_list ),
    ('ohot', OneHotEncoder( handle_unknown='infrequent_if_exist'),svc_oneh_list),
    ('has_1', HashingEncoder(n_components=100), 'settlement_new'),
    ('has_2' , HashingEncoder(n_components=20), 'region_new')
])

In [54]:
X_train = torch.from_numpy( preprocessing.fit_transform(X_train).astype(np.float32))
X_test  = torch.from_numpy(preprocessing.transform(X_test).astype(np.float32))

In [55]:
y_train = torch.tensor(y_train.to_numpy(), dtype=torch.float32)
y_test = torch.tensor( y_test.to_numpy(), dtype=torch.float32)                          

X_test.dtype, y_train.dtype

(torch.float32, torch.float32)

In [56]:
gc.collect()

54

In [57]:
class CustomDataset(Dataset):
    def __init__(self, data , target):
        self.data = data
        self.target = target
        

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.target[idx]

In [58]:
training_data = CustomDataset(X_train, y_train)
valid_data = CustomDataset(X_test, y_test)

print(len(training_data), X_train.shape)
print(len(valid_data), X_test.shape)

3230534 torch.Size([3230534, 235])
807634 torch.Size([807634, 235])


In [75]:
BATCH_SIZE = 4096

class_counts = torch.bincount(y_train.to(torch.int64))
weights = len(y_train)/class_counts.float()
sample_weights = weights[y_train.to(torch.int64)]

sampler = WeightedRandomSampler(weights=sample_weights, num_samples=len(sample_weights), replacement=True)
dataloader_train = DataLoader(training_data, batch_size=BATCH_SIZE) # sampler=sampler)

dataloader_valid  = DataLoader(valid_data, batch_size=9612, shuffle=True)

len(dataloader_train), dataloader_train.batch_size

(789, 4096)

In [82]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_size = 235, n=3,p=0.2,p_1=0.2):
        super().__init__()
        self.input_size = input_size
        self.n =n
        self.p = p
        self.p_1 = p_1
        self.fc1 = nn.Linear(input_size, self.n*input_size)
        self.fc2 = nn.Linear( self.n*input_size ,n*2*self.n*input_size)
        self.fc3 = nn.Linear(n*2*self.n*input_size, 512)
        self.fc4 = nn.Linear(512,128)
        self.fc5 = nn.Linear(128,64)
        self.fc6 = nn.Linear(64,1)
        
        self.drop = nn.Dropout(p)
        self.drop_1 = nn.Dropout(p_1)

    def forward(self, x):
        
        x = F.leaky_relu(self.fc1(x))
        x = self.drop(x)
        x = F.leaky_relu(self.fc2(x))
        x = F.leaky_relu(self.fc3(x))
        # x = self.drop_1(x)                  
        x = F.leaky_relu(self.fc4(x))
        x = F.leaky_relu(self.fc5(x))
       
        logits = self.fc6(x)

        return logits.squeeze()

In [78]:
# Экспереминтальная  архитектура
model = NeuralNetwork(X_train.shape[1],n=10, p=0.2756).to(device)
model

NeuralNetwork(
  (fc1): Linear(in_features=235, out_features=2350, bias=True)
  (fc2): Linear(in_features=2350, out_features=11750, bias=True)
  (fc3): Linear(in_features=11750, out_features=512, bias=True)
  (fc4): Linear(in_features=512, out_features=128, bias=True)
  (fc5): Linear(in_features=128, out_features=64, bias=True)
  (fc6): Linear(in_features=64, out_features=1, bias=True)
  (drop): Dropout(p=0.2756, inplace=False)
  (drop_1): Dropout(p=0.2, inplace=False)
)

In [64]:
next(model.parameters()).device

device(type='cuda', index=0)

In [65]:
def f1_score_custom(y_true, y_pred): 
    y_true = y_true.cpu().numpy()
    y_pred = (F.sigmoid(y_pred.detach())>0.5).cpu().int().numpy()
    f1=f1_score(y_true, y_pred)
    #print ('f1')
    return f1
    

In [3]:
def train_loop(dataloader, model, loss_fn, optimizer):
    
    num_batches = len(dataloader)
    total_loss = 0
    f1_score = 0
    
    model.train()

    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        pred = model(X)
        loss = loss_fn(pred, y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        f1_score += f1_score_custom(y,pred)
        total_loss+=loss.item()
        
       
    print(f'f1_score: {f1_score/num_batches:>0.8f}, train_loss: {total_loss/num_batches:8f} ')
    torch.cuda.empty_cache()
        

def test_loop(dataloader, model, loss_fn):
    
    num_batches = len(dataloader)
    total_loss = 0
    f1_score = 0   
    
    model.eval()
    with torch.no_grad():
        for batch, (X, y) in enumerate(dataloader):
            X, y = X.to(device), y.to(device)
            
            pred = model(X)
            loss = loss_fn(pred, y)
            
            f1_score += f1_score_custom(y,pred)
            total_loss+=loss.item()
        
    print(f'f1_score: {f1_score/num_batches:>0.8f}, test_loss: {total_loss/num_batches:8f} ')
    
    torch.cuda.empty_cache()
    return f1_score/num_batches


def train_model (epoch=100):
    LEANING_RATE = 1e-3
    loss_fn = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=LEANING_RATE,weight_decay=1e-5)  #,weight_decay=1e-4
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.1)

    
    for i in tqdm(range(epoch)):
        print(f"Epoch {i+1}\n-------------------------------")
        train_loop(dataloader_train, model, loss_fn, optimizer)
        test_loop(dataloader_valid, model, loss_fn)

        scheduler.step()

    

In [ ]:
def objective(trial):
    n = trial.suggest_int("n", 10, 20)  # Оптимизируем ширину слоев
    dropout_p = trial.suggest_float("dropout_p", 0.1, 0.5)  # Оптимизируем Dropout
    dropout_p_1 = trial.suggest_float("dropout_p_1", 0.1, 0.5)  # Оптимизируем Dropout
    weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-3, log=True)  # Оптимизируем weight_decay
    BATCH_SIZE = trial.suggest_int("batch_size", 1024, 9600, step=1024)
    
    model = NeuralNetwork(input_size=235, n=n, p=dropout_p, p_1=dropout_p).to(device)

    # BATCH_SIZE =9600

    class_counts = torch.bincount(y_train.to(torch.int64))
    weights = len(y_train)/class_counts.float()
    sample_weights = weights[y_train.to(torch.int64)]

    sampler = WeightedRandomSampler(weights=sample_weights, num_samples=len(sample_weights), replacement=True)
    dataloader_train = DataLoader(training_data, batch_size=BATCH_SIZE, sampler=sampler) # sampler=sampler)

    dataloader_valid  = DataLoader(valid_data, batch_size=BATCH_SIZE, shuffle=True)
      
    

    LEANING_RATE = 1e-3
    loss_fn = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=LEANING_RATE, weight_decay=weight_decay)  #,weight_decay=1e-4
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.1)

    EPOCH = 40
    for i in tqdm(range(EPOCH)):
        print(f"Epoch {i+1}\n-------------------------------")
        train_loop(dataloader_train, model, loss_fn, optimizer)
        f1_score = test_loop(dataloader_valid, model, loss_fn)

        scheduler.step()


    return f1_score  # Минимизируем 1 - F1
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=15)  

print(study.best_params,study.best_value)

[I 2025-02-15 20:36:26,710] A new study created in memory with name: no-name-4151b9fe-4f11-40d7-94f6-9cbb6f784421
/usr/local/lib/python3.10/dist-packages/optuna/distributions.py:699: UserWarning: The distribution is specified by [1024, 9600] and step=1024, but the range is not divisible by `step`. It will be replaced by [1024, 9216].
  warnings.warn(
  0%|          | 0/40 [00:00<?, ?it/s]

Epoch 1
-------------------------------
f1_score: 0.79715801, train_loss: 0.397690 


  2%|▎         | 1/40 [01:04<41:36, 64.01s/it]

f1_score: 0.81331888, test_loss: 0.321520 
Epoch 2
-------------------------------
f1_score: 0.82616718, train_loss: 0.343265 


  5%|▌         | 2/40 [02:08<40:47, 64.40s/it]

...

f1_score: 0.80835396, test_loss: 0.323616 
Epoch 2
-------------------------------
f1_score: 0.82663161, train_loss: 0.340433 


  5%|▌         | 2/40 [01:58<37:29, 59.19s/it]

f1_score: 0.81829471, test_loss: 0.309230 
Epoch 3
-------------------------------
f1_score: 0.82905703, train_loss: 0.335646 


  8%|▊         | 3/40 [02:57<36:30, 59.20s/it]

f1_score: 0.82038290, test_loss: 0.311336 
Epoch 4
-------------------------------
f1_score: 0.83049165, train_loss: 0.332772 


In [83]:
model = NeuralNetwork(X_train.shape[1],n=3, p=0.2756,p_1=0.1194).to(device)
model

NeuralNetwork(
  (fc1): Linear(in_features=235, out_features=705, bias=True)
  (fc2): Linear(in_features=705, out_features=4230, bias=True)
  (fc3): Linear(in_features=4230, out_features=512, bias=True)
  (fc4): Linear(in_features=512, out_features=128, bias=True)
  (fc5): Linear(in_features=128, out_features=64, bias=True)
  (fc6): Linear(in_features=64, out_features=1, bias=True)
  (drop): Dropout(p=0.2756, inplace=False)
  (drop_1): Dropout(p=0.1194, inplace=False)
)

Лучшие результаты были достигнуты с параметрами модели (n=3, p=0.2756, p_1=0.1194, weight_decay=4.55e-5, BATCH_SIZE=4096), где F1-скор на тестовых данных составил 0.8317. Однако, модель FFN не смогла превзойти результаты, полученные с использованием LGBM.